Các thành viên trong nhóm:
- Nguyễn Quang Huy
- Bùi Ngọc Sơn
- Nguyễn Thị Nhung

# IMPORTS

In [ ]:
!pip install requests-html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [ ]:
!pip install joblib

In [ ]:
from requests_html import HTMLSession
from datetime import datetime
from dateutil.relativedelta import relativedelta
import csv
import os
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import re
import joblib

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# CRAWL DATA FROM GOOGLE NEWS

In [ ]:
# function to crawl news data from google news
def get_news_articles(queries, start_date, end_date):
    base_url = "https://news.google.com/rss/search?q="
    date_format = "%Y-%m-%d"

    current_date = start_date
    total_articles = 0
    articles = []

    while current_date < end_date:
        next_date = current_date + relativedelta(weeks=1)
        formatted_start_date = current_date.strftime(date_format)
        formatted_end_date = next_date.strftime(date_format)

        for query in queries:
            url = f"{base_url}{query}+after:{formatted_start_date}+before:{formatted_end_date}"

            s = HTMLSession()
            r = s.get(url)

            for item in r.html.find('item'):
                title = item.find('title', first=True).text if item.find('title', first=True) else ''
                pubDate = item.find('pubDate', first=True).text if item.find('pubDate', first=True) else ''
                description_html = item.find('description', first=True).text if item.find('description', first=True) else ''
                soup = BeautifulSoup(description_html, 'html.parser')
                description = soup.find('a').text if soup.find('a') else ''

                articles.append({
                    'headline': title,
                    'date': pubDate,
                    'short_description': description,
                    'category': query.upper()
                })

                total_articles += 1

        current_date = next_date

    print(f'Total number of articles: {total_articles}')

    # Save articles to CSV
    with open('news_articles.csv', 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['headline', 'date', 'short_description', 'category']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for article in articles:
            writer.writerow(article)

In [ ]:
# 3 catgeroies, news from 01/01/2020 to current date
queries = ["war", "science", "entertainment"]
start_date = datetime(2020, 1, 1)
end_date = datetime.today()

In [ ]:
# call the function
articles = get_news_articles(queries, start_date, end_date)

/usr/lib/python3.10/html/parser.py:170: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


Total number of articles: 70517


# LOAD THE DATASET

In [ ]:
# load the dataset
dataset = pd.read_csv('/content/news_articles.csv')
dataset.drop(['date'], axis = 1, inplace = True)
dataset.head()

,headline,short_description,category
0,Photo Essay | Combat Veterans Tell Us What We ...,Photo Essay | Combat Veterans Tell Us What We ...,WAR
1,Are the U.S. and Iran Really on the Brink of W...,Are the U.S. and Iran Really on the Brink of War?,WAR
2,Mission Possible? Revisiting the ‘War on Cance...,Mission Possible? Revisiting the ‘War on Cance...,WAR
3,Operating in the Gray Zone: Countering Iran's ...,Operating in the Gray Zone: Countering Iran's ...,WAR
4,Fact check: Biden again dishonestly suggests h...,Fact check: Biden again dishonestly suggests h...,WAR


In [ ]:
# Calculate the number of unique categories in the 'category' column
cat_nums = dataset.category.nunique()

In [ ]:
# Display the count of each category in the 'category' column
print(f"We have a total of {cat_nums} categories")
dataset['category'].value_counts()

We have a total of 3 categories


category
ENTERTAINMENT    23591
WAR              23585
SCIENCE          23341
Name: count, dtype: int64

# PRE-PROCESSING

In [ ]:
# Create a copy of the dataset to avoid modifying the original data
df = dataset.copy()

In [ ]:
# Check for the number of duplicated rows in the entire dataframe
df.duplicated().sum()

9619

In [ ]:
# Drop duplicated rows, keeping the last occurrence
df.drop_duplicates(keep='last', inplace=True)

In [ ]:
# Check for duplicated rows based on 'short_description' and 'headline' columns
df.duplicated(subset=['short_description','headline']).sum()

43

In [ ]:
# Drop duplicates in 'short_description' and 'headline', keeping the last occurrence
df.drop_duplicates(subset=['short_description','headline'],keep='last',inplace=True)

In [ ]:
# Count the number of rows where 'headline' is empty
print(len(df[df['headline'] == ""]))

0


In [ ]:
# Replace empty 'headline' values with NaN and drop them
df.loc[df['headline'] == "", 'headline'] = np.nan
df.dropna(subset=['headline'], inplace=True)
print(len(df[df['headline'] == ""]))

0


In [ ]:
df.head()

,headline,short_description,category
1,Are the U.S. and Iran Really on the Brink of W...,Are the U.S. and Iran Really on the Brink of War?,WAR
2,Mission Possible? Revisiting the ‘War on Cance...,Mission Possible? Revisiting the ‘War on Cance...,WAR
3,Operating in the Gray Zone: Countering Iran's ...,Operating in the Gray Zone: Countering Iran's ...,WAR
4,Fact check: Biden again dishonestly suggests h...,Fact check: Biden again dishonestly suggests h...,WAR
5,US: Trump Threatens War Crimes Against Iran - ...,US: Trump Threatens War Crimes Against Iran,WAR


In [ ]:
# Count the number of rows where 'short_description' is empty
print(len(df[df['short_description'] == ""]))

0


In [ ]:
# Replace empty 'short_description' values with NaN and drop them
df.loc[df['short_description'] == "", 'short_description'] = np.nan
df.dropna(subset=['short_description'], inplace=True)
print(len(df[df['short_description'] == ""]))

0


In [ ]:
df.head()

,headline,short_description,category
1,Are the U.S. and Iran Really on the Brink of W...,Are the U.S. and Iran Really on the Brink of War?,WAR
2,Mission Possible? Revisiting the ‘War on Cance...,Mission Possible? Revisiting the ‘War on Cance...,WAR
3,Operating in the Gray Zone: Countering Iran's ...,Operating in the Gray Zone: Countering Iran's ...,WAR
4,Fact check: Biden again dishonestly suggests h...,Fact check: Biden again dishonestly suggests h...,WAR
5,US: Trump Threatens War Crimes Against Iran - ...,US: Trump Threatens War Crimes Against Iran,WAR


In [ ]:
# Shuffle the dataframe rows randomly and reset the index
df = shuffle(df)
df.reset_index(inplace=True, drop=True)

In [ ]:
df.head()

,headline,short_description,category
0,Political Science and International Affairs of...,Political Science and International Affairs of...,SCIENCE
1,How Big are the Big Four? - Statista,How Big are the Big Four?,ENTERTAINMENT
2,"Rebecca Rittenhouse talks 'Maggie,' those prom...","Rebecca Rittenhouse talks 'Maggie,' those prom...",ENTERTAINMENT
3,Can former conservation ‘pirates’ help scienti...,Can former conservation ‘pirates’ help scienti...,SCIENCE
4,A ‘lake’ on Mars may be surrounded by more poo...,A ‘lake’ on Mars may be surrounded by more poo...,SCIENCE


In [ ]:
# Combine 'headline' and 'short_description' into a new column 'desc'
df['desc'] = df['headline'].astype(str)+"-"+df['short_description']
# Drop the original 'headline' and 'short_description' columns
df.drop(columns =['headline','short_description'],axis = 1, inplace=True)
df.astype(str)
df.head()

,category,desc
0,SCIENCE,Political Science and International Affairs of...
1,ENTERTAINMENT,How Big are the Big Four? - Statista-How Big a...
2,ENTERTAINMENT,"Rebecca Rittenhouse talks 'Maggie,' those prom..."
3,SCIENCE,Can former conservation ‘pirates’ help scienti...
4,SCIENCE,A ‘lake’ on Mars may be surrounded by more poo...


In [ ]:
# Convert the text in the 'desc' column to lowercase
df['desc'] = df['desc'].apply(lambda x: x.lower())
df.head()

,category,desc
0,SCIENCE,political science and international affairs of...
1,ENTERTAINMENT,how big are the big four? - statista-how big a...
2,ENTERTAINMENT,"rebecca rittenhouse talks 'maggie,' those prom..."
3,SCIENCE,can former conservation ‘pirates’ help scienti...
4,SCIENCE,a ‘lake’ on mars may be surrounded by more poo...


In [ ]:
# Remove stopwords from the 'desc' column
stop = set(stopwords.words('english'))
df['desc'] = df['desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop]))
df.head()

,category,desc
0,SCIENCE,political science international affairs offers...
1,ENTERTAINMENT,big big four? - statista-how big big four?
2,ENTERTAINMENT,"rebecca rittenhouse talks 'maggie,' prom looks..."
3,SCIENCE,former conservation ‘pirates’ help scientists ...
4,SCIENCE,‘lake’ mars may surrounded pools water - scien...


In [ ]:
# Remove non-word characters from the 'desc' column
df['desc'] = df['desc'].apply(lambda x: re.sub(r'\W', ' ', str(x)))
df.head()

,category,desc
0,SCIENCE,political science international affairs offers...
1,ENTERTAINMENT,big big four statista how big big four
2,ENTERTAINMENT,rebecca rittenhouse talks maggie prom looks...
3,SCIENCE,former conservation pirates help scientists ...
4,SCIENCE,lake mars may surrounded pools water scien...


In [ ]:
# Apply stemming to the words in the 'desc' column
stemmer = PorterStemmer()

df['desc'] = df['desc'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
df.head()

,category,desc
0,SCIENCE,polit scienc intern affair offer three fulli o...
1,ENTERTAINMENT,big big four statista how big big four
2,ENTERTAINMENT,rebecca rittenhous talk maggi prom look potent...
3,SCIENCE,former conserv pirat help scientist studi ocea...
4,SCIENCE,lake mar may surround pool water scienc news m...


# DATA SPLITTING

In [ ]:
X, Y = df['desc'],df['category']

# Split 80% to train , 10% for validation , 10% for testing
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
X_val, X_test , y_val, y_test= train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [ ]:
# Set parameters for tokenization and padding
vocab_size = 20000       # Maximum number of words to keep based on word frequency
max_length = 150         # Maximum length of all sequences
trunc_type = 'post'      # Truncate sequences after the max_length
padding_type = 'post'    # Pad sequences after the max_length
oov_tok = "<OOV>"        # Token for out-of-vocabulary words

In [ ]:
# Initialize the Tokenizer with the given vocab size and OOV token
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

# Fit the tokenizer on the training data to create the word index
tokenizer.fit_on_texts(X_train)

# Retrieve the word index created by the tokenizer
word_index = tokenizer.word_index

# Convert training text data to sequences and pad them
X_train = tokenizer.texts_to_sequences(X_train)
X_train = pad_sequences(X_train, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert validation text data to sequences and pad them
X_val = tokenizer.texts_to_sequences(X_val)
X_val = pad_sequences(X_val, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Save the original test data for later use
X_test_words = X_test

# Convert test text data to sequences and pad them
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Convert the padded sequences to numpy arrays
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)

# Initialize LabelEncoder for converting labels to integers
le = LabelEncoder()

# Fit the label encoder on the training labels and transform them
y_train = le.fit_transform(y_train)
y_val = le.fit_transform(y_val)
y_test = le.fit_transform(y_test)

# Convert the integer labels to one-hot encoded format
y_train = tf.keras.utils.to_categorical(y_train, num_classes=cat_nums)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=cat_nums)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=cat_nums)

# Print the shapes of the train, validation, and test datasets
print(X_train.shape)
print(y_train.shape)

print(X_val.shape)
print(y_val.shape)

print(X_test.shape)
print(y_test.shape)

(48684, 150)
(48684, 3)
(6085, 150)
(6085, 3)
(6086, 150)
(6086, 3)


# EMBEDDING

In [ ]:
path_to_glove_file =  'glove.6B.100d.txt'
if not os.path.exists(path_to_glove_file):
  !wget http://nlp.stanford.edu/data/glove.6B.zip #downloadingu glove vec word embeddings
  !unzip -q glove.6B.zip #unzipping

--2024-07-26 12:40:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-07-26 12:40:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-07-26 12:40:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
#Initialising the embedding matrix with glove vec embeddings
num_tokens = len(tokenizer.word_index.items()) + 2
embedding_dim = 100
hits = 0
misses = 0

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Found 400000 word vectors.
Converted 18389 words (9976 misses)


# LSTM MODEL

In [ ]:
# Define an early stopping callback to stop training when the validation loss stops improving
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    min_delta=0.0001     # Minimum change in the monitored quantity to qualify as an improvement
)

In [ ]:
# Define the embedding size
embed_size = 100        # glove.6B.100d.txt

# Build the Sequential model
model = Sequential([
    # Add an Embedding layer with pre-trained embeddings
    Embedding(
        num_tokens,               # Vocabulary size
        embedding_dim,            # Dimension of the embedding vectors
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        mask_zero=True,           # Mask padding tokens (0) in sequences
        input_shape=[None],       # Input shape for the embedding layer
        trainable=False           # Freeze the embedding layer to keep the pre-trained weights
    ),
    # Add a Bidirectional LSTM layer with dropout
    Bidirectional(LSTM(256, dropout=0.4)),
    # Add a Dense output layer with softmax activation for multi-class classification
    Dense(y_train.shape[1], activation="softmax")
])

In [ ]:
# Print the model summary to see the architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         2836700   
                                                                 
 bidirectional (Bidirection  (None, 512)               731136    
 al)                                                             
                                                                 
 dense (Dense)               (None, 3)                 1539      
                                                                 
Total params: 3569375 (13.62 MB)
Trainable params: 732675 (2.79 MB)
Non-trainable params: 2836700 (10.82 MB)
_________________________________________________________________


In [ ]:
# Define the optimizer with a specified learning rate
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model with categorical crossentropy loss and accuracy metric
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
# Train the model on the training data, with early stopping based on validation loss
history = model.fit(
    X_train, y_train,
    batch_size=32,                # Number of samples per batch
    validation_data=(X_val, y_val),  # Validation data
    epochs=100,                   # Maximum number of epochs to train
    callbacks=[early_stop]        # Include the early stopping callback
)

Epoch 1/100
1522/1522 [==============================] - 2045s 1s/step - loss: 0.1503 - accuracy: 0.9494 - val_loss: 0.1132 - val_accuracy: 0.9634
Epoch 2/100
1522/1522 [==============================] - 2001s 1s/step - loss: 0.1058 - accuracy: 0.9650 - val_loss: 0.0989 - val_accuracy: 0.9691
Epoch 3/100
1522/1522 [==============================] - 1978s 1s/step - loss: 0.0927 - accuracy: 0.9694 - val_loss: 0.0967 - val_accuracy: 0.9701
Epoch 4/100
1522/1522 [==============================] - 1956s 1s/step - loss: 0.0804 - accuracy: 0.9736 - val_loss: 0.0873 - val_accuracy: 0.9732
Epoch 5/100
1522/1522 [==============================] - 1936s 1s/step - loss: 0.0691 - accuracy: 0.9769 - val_loss: 0.0939 - val_accuracy: 0.9727
Epoch 6/100
1522/1522 [==============================] - 1931s 1s/step - loss: 0.0609 - accuracy: 0.9794 - val_loss: 0.0888 - val_accuracy: 0.9739
Epoch 7/100
1522/1522 [==============================] - 1956s 1s/step - loss: 0.0492 - accuracy: 0.9841 - val_loss: 0

In [ ]:
# Save the trained model to a file
model.save('lstm.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#making predictions
preds = model.predict(X_test,verbose = 1)
#converting the one hot vector output to a linear numpy array.
pred_classes = np.argmax(preds, axis = 1)

191/191 [==============================] - 76s 364ms/step


In [ ]:
#extracting the classes from the label encoder
encoded_classes = le.classes_
#mapping the encoded output to actual categories
predicted_category = [encoded_classes[x] for x in pred_classes]
true_category = [encoded_classes[np.argmax(x)] for x in y_test]

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(true_category, predicted_category)
print(f"Accuracy: {accuracy}")

# Calculate precision, recall, f1-score
precision = precision_score(true_category, predicted_category, average='weighted')
recall = recall_score(true_category, predicted_category, average='weighted')
f1 = f1_score(true_category, predicted_category, average='weighted')

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")

# Detailed classification report
report = classification_report(true_category, predicted_category)
print("Classification Report:\n", report)

Accuracy: 0.9773250082155768
Precision: 0.9774611962394947
Recall: 0.9773250082155768
F1-Score: 0.9773552453539662
Classification Report:
                precision    recall  f1-score   support

ENTERTAINMENT       0.98      0.97      0.98      2012
      SCIENCE       0.96      0.98      0.97      2051
          WAR       0.99      0.98      0.98      2023

     accuracy                           0.98      6086
    macro avg       0.98      0.98      0.98      6086
 weighted avg       0.98      0.98      0.98      6086



# RANDOM FORREST

In [ ]:
# Flatten the sequences to convert them into a 2D array suitable for traditional ML models
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

In [ ]:
# Initialize a Random Forest classifier with 100 trees and a fixed random state for reproducibility
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model using the flattened training data
rf_model.fit(X_train_flat, y_train)

# Predict labels for the test set using the trained Random Forest model
y_test_pred_rf = rf_model.predict(X_test_flat)

# Evaluate the model's performance using various metrics
accuracy_rf = accuracy_score(y_test, y_test_pred_rf)                   # Calculate accuracy
precision_rf = precision_score(y_test, y_test_pred_rf, average='weighted')  # Calculate weighted precision
recall_rf = recall_score(y_test, y_test_pred_rf, average='weighted')       # Calculate weighted recall
f1_rf = f1_score(y_test, y_test_pred_rf, average='weighted')               # Calculate weighted F1-score

# Print a detailed classification report for the Random Forest model
print("Random Forest Classification Report:")
print(classification_report(y_test, y_test_pred_rf))
print(f"Accuracy: {accuracy_rf}")
print(f"Precision: {precision_rf}")
print(f"Recall: {recall_rf}")
print(f"F1-Score: {f1_rf}")

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.80      0.87      2012
           1       0.99      0.81      0.89      2051
           2       0.99      0.86      0.92      2023

   micro avg       0.98      0.83      0.89      6086
   macro avg       0.98      0.83      0.89      6086
weighted avg       0.98      0.83      0.89      6086
 samples avg       0.83      0.83      0.83      6086

Accuracy: 0.8256654617154124
Precision: 0.9762131977229076
Recall: 0.8256654617154124
F1-Score: 0.8945428611213516


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
joblib.dump(rf_model, 'random_forest_model.pkl')

['random_forest_model.pkl']

# K-Nearest Neighbors (KNN)

In [ ]:
# Initialize a K-Nearest Neighbors classifier with 5 neighbors
knn_model = KNeighborsClassifier(n_neighbors=5)

# Train the KNN model using the flattened training data
knn_model.fit(X_train_flat, y_train)

# Predict labels for the test set using the trained KNN model
y_test_pred_knn = knn_model.predict(X_test_flat)

# Evaluate the model's performance using various metrics
accuracy_knn = accuracy_score(y_test, y_test_pred_knn)                      # Calculate accuracy
precision_knn = precision_score(y_test, y_test_pred_knn, average='weighted')  # Calculate weighted precision
recall_knn = recall_score(y_test, y_test_pred_knn, average='weighted')       # Calculate weighted recall
f1_knn = f1_score(y_test, y_test_pred_knn, average='weighted')               # Calculate weighted F1-score

# Print a detailed classification report for the KNN model
print("K-Nearest Neighbors Classification Report:")
print(classification_report(y_test, y_test_pred_knn))
print(f"Accuracy: {accuracy_knn}")
print(f"Precision: {precision_knn}")
print(f"Recall: {recall_knn}")
print(f"F1-Score: {f1_knn}")

K-Nearest Neighbors Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.33      0.40      2012
           1       0.44      0.28      0.34      2051
           2       0.45      0.41      0.43      2023

   micro avg       0.46      0.34      0.39      6086
   macro avg       0.47      0.34      0.39      6086
weighted avg       0.47      0.34      0.39      6086
 samples avg       0.34      0.34      0.34      6086

Accuracy: 0.3411107459743674
Precision: 0.4669807259225424
Recall: 0.3411107459743674
F1-Score: 0.3913513250190275


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
joblib.dump(knn_model, 'knn_model.pkl')

['knn_model.pkl']